In [1]:
import scanpy as sc
import scvelo as scv
import anndata
import pandas as pd
import cellrank as cr
import numpy as np
import scachepy

np.random.seed(42)

scv.settings.verbosity = 3
scv.settings.set_figure_params('scvelo', frameon=True)
cr.settings.verbosity = 2

In [2]:
adata_w_velo = scv.read("./after_cellrank.h5ad")

In [3]:
import scipy.io
from scipy.sparse import csr_matrix
scipy.io.mmwrite('endo_expression_Ms.mtx', csr_matrix(adata_w_velo.layers['Ms']))

In [4]:
scv.tl.recover_latent_time(
    adata_w_velo, root_key="initial_states_probs", end_key="terminal_states_probs"
)

computing latent time using initial_states_probs, terminal_states_probs as prior
    finished (0:01:31) --> added 
    'latent_time', shared time (adata.obs)


In [5]:
adata_lite = anndata.AnnData(X=None,
                obs = adata_w_velo.obs,
                var = adata_w_velo.var,
                uns = adata_w_velo.uns,
                obsm = adata_w_velo.obsm,
                varm = adata_w_velo.varm)

In [6]:
keys = ['to_terminal_states_names', 'from_initial_states_names', 'initial_states_names']
out_dict = {}
for key in keys:
    out_dict[key] = adata_lite.uns[key]
    
print(out_dict)
adata_lite.uns = out_dict

{'to_terminal_states_names': array(['miEC_1', 'Lymphatic_1', 'Lymphatic_2', 'Arterial maEC',
       'Venous maEC', 'miEC_2', 'Proliferating\nmiEC', 'Car4+', 'miEC_3'],
      dtype=object), 'from_initial_states_names': array(['Lymphatic', 'Proliferating\nmiEC', 'Arterial maEC', 'miEC',
       'Car4+'], dtype=object), 'initial_states_names': array(['Lymphatic', 'Proliferating\nmiEC', 'Arterial maEC', 'miEC',
       'Car4+'], dtype=object)}


In [7]:
adata_lite.write("./only_metadata.h5ad", compression = 'gzip')